In [37]:
import os
from shutil import copyfile
import cv2
import base64
import numpy as np
import glob
import json
import pickle as pkl
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

##  image_augumentation_json_file_4_points

In [27]:

def rotate_im(image, angle):

    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)

    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    image = cv2.warpAffine(image, M, (nW, nH))

#    image = cv2.resize(image, (w,h))
    return image



def rotate_box(corners,angle,  cx, cy, h, w):
    
    corners = corners.reshape(-1,2)
    corners = np.hstack((corners, np.ones((corners.shape[0],1), dtype = type(corners[0][0]))))
    
    M = cv2.getRotationMatrix2D((cx, cy), angle, 1.0)
    
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    M[0, 2] += (nW / 2) - cx
    M[1, 2] += (nH / 2) - cy
    calculated = np.dot(M,corners.T).T
    
    calculated = calculated.reshape(-1,4)
    
    return calculated

def Rotation(img,bboxes,angle):
    
    if type(angle) == tuple:
        assert len(angle) == 2, "Invalid range"  
    else:
        angle = (-angle, angle)

    angle = random.uniform(*angle)
    
    w,h = img.shape[1], img.shape[0]
    cx, cy = w//2, h//2

    img = rotate_im(img, angle)

    corners = bboxes

    corners[:,:4] = rotate_box(corners[:,:4], angle, cx, cy, h, w)

    new_bbox = corners

    scale_factor_x = img.shape[1] / w

    scale_factor_y = img.shape[0] / h

    img = cv2.resize(img, (w,h))

    new_bbox[:,:] /= [scale_factor_x, scale_factor_y, scale_factor_x, scale_factor_y] 

    bboxes  = new_bbox

    return img, bboxes



def horizontalFlip(img,bboxes):
    
    img_center = np.array(img.shape[:2])[::-1]/2
    img_center = np.hstack((img_center, img_center))
    
    img = img[:, ::-1, :]
    bboxes[:, [0, 2]] += 2*(img_center[[0, 2]] - bboxes[:, [0, 2]])

    box_w = abs(bboxes[:, 0] - bboxes[:, 2])

    bboxes[:, 0] -= box_w
    bboxes[:, 2] += box_w

    
    return img, bboxes


def shear(img,bboxes,shear_factor):
    
    if type(shear_factor) == tuple:
            assert len(shear_factor) == 2, "Invalid range for scaling factor"   
    else:
        shear_factor = (-shear_factor, shear_factor)
        
    shear_factor = random.uniform(*shear_factor)
    
    w,h = img.shape[1], img.shape[0]

    if shear_factor < 0:
        img, bboxes = horizontalFlip(img, bboxes)

    M = np.array([[1, abs(shear_factor), 0],[0,1,0]])

    nW =  img.shape[1] + abs(shear_factor*img.shape[0])

    bboxes[:,[0,2]] += ((bboxes[:,[1,3]]) * abs(shear_factor) ).astype(int) 

    img = cv2.warpAffine(img, M, (int(nW), img.shape[0]))

    if shear_factor < 0:
        img, bboxes = horizontalFlip(img, bboxes)

    img = cv2.resize(img, (w,h))

    scale_factor_x = nW / w
    
    bboxes[:,:] /= [[scale_factor_x, 1, scale_factor_x, 
                     1] for b in range(len(bboxes)) ]


    return img, bboxes



In [48]:
for file_name in glob.glob('inputs/*.json'):
    
    image=cv2.imread(file_name.replace('.json','.jpg'))
    copy_image=image.copy()
    
    with open(file_name,'r') as file:
        dicts=json.loads(file.read())
    
    number_of_objects=len(dicts['shapes']) 
    
    objects_co_ordinates=[]
    
    for value in range(number_of_objects):
        
        startX=(dicts['shapes'][0]['points'][0][0])
        startY=(dicts['shapes'][0]['points'][0][1])
        endX=(dicts['shapes'][0]['points'][1][0])
        endY=(dicts['shapes'][0]['points'][1][1])
        cv2.rectangle(image,(int(startX),int(startY)),(int(endX),int(endY)),(255,0,0),2)
        cv2.imwrite('input.png',image)               


        object_co_ordinate=[startX,startY,endX,endY]        
        objects_co_ordinates.append(object_co_ordinate)
    
    objects_co_ordinates=np.array(objects_co_ordinates)

    image,co_ordination=Rotation(image,objects_co_ordinates,30)
    cv2.imwrite('augumentation_input.png',image)
    
#     input_image=cv2.imread('ugumentation_input.png')
#     cv2.rectangle(input_image,(int(co_ordination[0][0]),int(co_ordination[0][1])),(int(co_ordination[0][2]),int(co_ordination[0][3])),(255,0,0),2)
#     cv2.imwrite('output.png',input_image)
    
    break
    

                      

In [119]:
for file_name in glob.glob('inputs/*.json'):
    
    image=cv2.imread(file_name.replace('.json','.jpg'))
    
    with open(file_name,'r') as file:
        dicts=json.loads(file.read())
    
    number_of_objects=len(dicts['shapes']) 
    
    objects_co_ordinates=[]
    
    for value in range(number_of_objects):
        
        startX=dicts['shapes'][0]['points'][0][0]
        startY=dicts['shapes'][0]['points'][0][1]
        endX=dicts['shapes'][0]['points'][1][0]
        endY=dicts['shapes'][0]['points'][1][1]
        cv2.rectangle(image,(int(startX,startY),int(endX,endY),(255,0,0),2)
        plt.imshow(image)
        cv2.imwrite('input.png',image)

        object_co_ordinate=[startX,startY,endX,endY]
        
        objects_co_ordinates.append(object_co_ordinate)
        
    objects_co_ordinates=np.array(objects_co_ordinates)
    
    Total_iteration=[1,2,3,4]
    Total_iteration=random.choice(Total_iteration)
    
    
    for iteration in range(Total_iteration):
        
        random_choice=['Rotation','horizontalFlip','shear']
        random_choice=random.choice(random_choice)
        
        if random_choice=='Rotation':
            image,co_ordination=Rotation(image,objects_co_ordinates,90)
            starX=int(co_ordination[0][0])
            starY=int(co_ordination[0][1])
            endX=int(co_ordination[0][2])
            endY=int(co_ordination[0][3])
            cv2.rectangle(image,(starX,starY),(endX,endY),(255,0,0),2)
#             plt.imshow(image)
#             cv2.imwrite('output.png',image)
#                      

        if random_choice=='horizontalFlip':
            image,co_ordination=horizontalFlip(image,objects_co_ordinates)

        if random_choice=='shear':
            image,co_ordination=shear(image,objects_co_ordinates,0.3)
                
    bbox=co_ordination
        
    for i in range(len(bbox)):

        startX=bbox[0][0]
        startY=bbox[0][1]
        endX=bbox[0][2]
        endY=bbox[0][3]

        dicts['shapes'][i]['points'][0][0]=startX
        dicts['shapes'][i]['points'][0][0]=startY
        dicts['shapes'][i]['points'][0][0]=endX
        dicts['shapes'][i]['points'][0][0]=endY

    output_image_name=file_name.split('/')[-1].split('.')[0]

    cv2.imwrite('output/'+output_image_name+'_augumentation_image_'+str(iteration)+'.jpg',image)

    json_file_name=file_name.split('/')[-1].split('.')[0]
    
    encoding_image=json_file_name+'_augumentation_image_'+str(iteration)+'.jpg'
    
    
    with open('output/'+encoding_image,'rb') as file:
        reading=base64.b64encode(file.read())
        dicts['imageData']=reading.decode('utf-8')
    
    
    output_json_file=encoding_image.split('.')[0]+'.json'
    
    with open('output/'+output_json_file,'w') as file:
        json.dump(dicts,file)
        
    break

SyntaxError: invalid syntax (<ipython-input-119-1c6129ffed90>, line 19)

In [25]:

for i in tqdm(glob.glob('inputs/*.json')):
    
    try:
        img = cv2.imread(i.replace('.json','.jpg'))   
        
        with open(i,'r') as f:
            dicts = json.loads(f.read())

            bboxes = []
        for shape_index in range(len(dicts['shapes'])):

            x_min = dicts['shapes'][shape_index]['points'][0][0]
            y_min = dicts['shapes'][shape_index]['points'][0][1]
            x_max = dicts['shapes'][shape_index]['points'][2][0]
            y_max = dicts['shapes'][shape_index]['points'][2][1]
            x_max_left = dicts['shapes'][shape_index]['points'][1][0]
            y_max_left = dicts['shapes'][shape_index]['points'][1][1]
            x_min_right = dicts['shapes'][shape_index]['points'][3][0]
            y_min_right = dicts['shapes'][shape_index]['points'][3][1]

            bboxes_tmp = [x_min,y_min,x_max,y_max,x_max_left,y_max_left,x_min_right,y_min_right]
            bboxes.append(bboxes_tmp)
        bboxes = np.asarray(bboxes)

        Total_iteration=[1,2,3,4]
        Total_iteration=random.choice(Total_iteration)

        img_ = img
        bboxes_ = bboxes
        Count=[1,2,3,4,5]
        print(type(bboxes_),'box type')
        print('boxes value :-',bboxes_,bboxes_.shape)

        for row in range(Total_iteration):

            agumentation_list=['Shear','Rotation','Translate']
            choice=random.choice(agumentation_list)

            if(choice=='Shear'):
                img_,bboxes_ = shear(img_,bboxes_,0.3)
    #             print('Horizontal Flip is Completed')
    #             plotted_img = draw_rect(img_, bboxes_)
    #             plt.imshow(plotted_img)
    #             plt.show()

            elif(choice=='Rotation'):
                img_, bboxes_ = Rotation(img_,bboxes_,20)
    #             plotted_img = draw_rect(img_, bboxes_)
    #             print('Rotation is Completed')
    #             plt.imshow(plotted_img)
    #             plt.show()


            elif(choice=='Translate'):
                Total_Number_Translate=random.choice(Count)
                for row in range (Total_Number_Translate):
                    img_, bboxes_=Translate(img_,bboxes_,0.3,True)
    #                 plotted_img = draw_rect(img_, bboxes_)
    #                 print('Translate is Completed')
    #                 plt.imshow(plotted_img)
    #                 plt.show()

        for k in range(len(bboxes_)):
            x_min = bboxes_[k][0]
            y_min = bboxes_[k][1]
            x_max = bboxes_[k][2]
            y_max = bboxes_[k][3]
            x_max_left =   bboxes_[k][4]
            y_max_left =   bboxes_[k][5]
            x_min_right =   bboxes_[k][6]
            y_min_right =   bboxes_[k][7]

            # Updating Current Cooridinate Value into Dictionary:-
            dicts['shapes'][k]['points'][0][0] = x_min
            dicts['shapes'][k]['points'][0][1] = y_min
            dicts['shapes'][k]['points'][2][0] = x_max
            dicts['shapes'][k]['points'][2][1] = y_max
            dicts['shapes'][k]['points'][1][0] = x_max_left
            dicts['shapes'][k]['points'][1][1] = y_max_left
            dicts['shapes'][k]['points'][3][0] = x_min_right
            dicts['shapes'][k]['points'][3][1] = y_min_right
        name = random.randint(0,1000)
        name_2 = random.randint(0,1000)

        # Storing Image:-
        output=i
        output=output.split('/')
        last_one=output[-1]
        output_file_name=last_one.replace('.json','.jpg')
        print('write operation')
        cv2.imwrite('outputs/'+ output_file_name.replace('.jpg','')+'_'+str(name)+'_'+str(name_2)+'_augmented'+'.jpg',img_)

        # Target image Size:-
        targetSize_x=img_.shape[0]
        targetSize_y=img_.shape[1]

        # Update Heihgt and Weight into Dictionary:-
        dicts['imageHeight'] = targetSize_x
        dicts['imageWidth'] = targetSize_y

        # Stroing JSON File:-
    #     object_name_agument_number=
        with open('outputs/'+ output_file_name.replace('.jpg','')+'_'+str(name)+'_'+str(name_2)+'_augmented'+'.jpg', "rb") as image_file: 
            my_string = base64.b64encode(image_file.read())
        dicts['imageData'] = my_string.decode('utf-8')
        dicts['imagePath'] = output_file_name.replace('.jpg','')+'_'+str(name)+'_'+str(name_2)+'_augmented'+'.jpg'
        # Write Updated Dictionary to the JSON File:-
        output_file_name=last_one.replace('.jpg','.json')
        with open('outputs/'+output_file_name.replace('.json','')+'_'+str(name)+'_'+str(name_2)+'_augmented'+'.json','w') as f:
            json.dump(dicts,f)
            
    except:
        pass


100%|██████████| 3/3 [00:00<00:00, 48.24it/s]

(678, 1200, 3)
(678, 1200, 3)
(600, 1019, 3)
1
Total_iteration is:-> 4
<class 'numpy.ndarray'> box type
boxes value :- [[1014.35483871    4.39784946    1.4516129   594.72043011 1014.35483871
   595.79569892    1.4516129     3.32258065]] (1, 8)
write operation


In [184]:
from tkinter import *  
  
top = Tk()  
  
top.geometry("400x250")  
  
name='arun'

uname = Label(top, text = "welcome "+name).place(x = 150,y = 5)  
password = Label(top, text = "Enter Your Amount").place(x = 10, y = 90)  
e1 = Entry(top,width = 20).place(x = 150, y = 90)  
  
top.mainloop()

In [185]:
from tkinter import *
from tkinter.ttk import *

# creates a Tk() object
master = Tk()

master.geometry("400x250")  

name='arun'

password = Label(master, text = "Enter Your Amount").place(x = 10, y = 90)  

e1 = Entry(master,width = 20).place(x = 150, y = 90)  

def openNewWindow():
    newWindow = Toplevel(master)
    newWindow.title("New Window")
    newWindow.geometry("400x250")
    Label(newWindow,text ="sucess").pack()

label = Label(master,text ="welcome "+name)

label.pack(pady = 10)

btn = Button(master,text ="ok",command = openNewWindow)
btn.pack(pady = 10)
mainloop()


In [3]:
from tkinter import messagebox

import tkinter as tk

root=tk.Tk()
root.geometry("600x400")

name_var=tk.StringVar()
passw_var=tk.StringVar()

name='arun'

def Close():
    root.destroy()

def openNewWindow():
    
    name=int(name_var.get())
        
    if name>1000:
#         newWindow = Toplevel(root)
        newWindow=tk.Toplevel(root)
        newWindow.title("New Window")
        newWindow.geometry("400x250")
        sub_btn=tk.Button(newWindow,text = 'Exit',command=Close)
        sub_btn.pack(pady=10)
        
    else:
        messagebox.showwarning("showwarning", "Warning")
        root.destroy()

    
name_label = tk.Label(root, text = 'Enter Your Amount', font=('calibre',10, 'bold'),anchor='nw')
name_entry = tk.Entry(root,textvariable = name_var, font=('calibre',10,'normal'))
sub_btn=tk.Button(root,text = 'ok', command = openNewWindow)
name_label.grid(row=0,column=0)
name_entry.grid(row=0,column=1)
sub_btn.grid(row=3,column=1)
root.mainloop()
